In [2]:
%cd ..

/home/arun/mtp/MPNN-Ptr


In [3]:
import torch
from models.mpnn_ptr import MpnnPtr
from torch_geometric.loader import DataLoader
from utils.utils import communication_cost_multiple_samples
from utils.datagenerate import generate_distance_matrix
import numpy as np
import math

In [2]:
mpnn_ptr = MpnnPtr(input_dim=16, embedding_dim=16 + 10, hidden_dim=16 + 20, K=3, n_layers=2, p_dropout=0.1, device=torch.device('cuda'), logit_clipping=True, decoding_type='greedy',feature_scale=290)

In [3]:
mpnn_ptr.load_state_dict(torch.load('./models_data_final/model_16_01-10.pt',map_location=torch.device('cuda')))
mpnn_ptr.to(torch.device('cuda'))

MpnnPtr(
  (mpnn): Mpnn(
    (node_init_embedding_layer): Sequential(
      (0): Linear(in_features=16, out_features=26, bias=False)
      (1): ReLU()
    )
    (theta4layers): ModuleList(
      (0): Sequential(
        (0): Linear(in_features=52, out_features=26, bias=False)
        (1): ReLU()
      )
      (1): Sequential(
        (0): Linear(in_features=52, out_features=26, bias=False)
        (1): ReLU()
      )
      (2): Sequential(
        (0): Linear(in_features=52, out_features=26, bias=False)
        (1): ReLU()
      )
    )
    (theta5layers): ModuleList(
      (0): Sequential(
        (0): Linear(in_features=52, out_features=26, bias=False)
        (1): ReLU()
      )
      (1): Sequential(
        (0): Linear(in_features=52, out_features=26, bias=False)
        (1): ReLU()
      )
      (2): Sequential(
        (0): Linear(in_features=52, out_features=26, bias=False)
        (1): ReLU()
      )
    )
    (theta6): Sequential(
      (0): Linear(in_features=26, out_feature

In [4]:
graph_data = torch.load('./data_tgff/data_single_TGFF1_16.pt', map_location=torch.device('cuda'))
datalist = [graph_data]
dataloader = DataLoader(datalist, batch_size = 1)
data =  next(iter(dataloader))
data

DataBatch(x=[16, 16], edge_index=[2, 21], edge_attr=[21, 1], batch=[16], ptr=[2])

In [5]:
n = math.ceil(math.sqrt(graph_data.num_nodes))
m = math.ceil(graph_data.num_nodes / n)
distance_matrix = generate_distance_matrix(n,m).to(torch.device('cuda'))

In [6]:
num_samples = 10000

In [7]:
mpnn_ptr.eval()

MpnnPtr(
  (mpnn): Mpnn(
    (node_init_embedding_layer): Sequential(
      (0): Linear(in_features=16, out_features=26, bias=False)
      (1): ReLU()
    )
    (theta4layers): ModuleList(
      (0): Sequential(
        (0): Linear(in_features=52, out_features=26, bias=False)
        (1): ReLU()
      )
      (1): Sequential(
        (0): Linear(in_features=52, out_features=26, bias=False)
        (1): ReLU()
      )
      (2): Sequential(
        (0): Linear(in_features=52, out_features=26, bias=False)
        (1): ReLU()
      )
    )
    (theta5layers): ModuleList(
      (0): Sequential(
        (0): Linear(in_features=52, out_features=26, bias=False)
        (1): ReLU()
      )
      (1): Sequential(
        (0): Linear(in_features=52, out_features=26, bias=False)
        (1): ReLU()
      )
      (2): Sequential(
        (0): Linear(in_features=52, out_features=26, bias=False)
        (1): ReLU()
      )
    )
    (theta6): Sequential(
      (0): Linear(in_features=26, out_feature

In [8]:
torch. set_printoptions(profile="full")
mpnn_ptr.decoding_type = 'greedy'
mappings, log_probs = mpnn_ptr(data, num_samples=num_samples)
costs = communication_cost_multiple_samples(data.edge_index, data.edge_attr, data.batch, distance_matrix, mappings, num_samples=num_samples)
_,indices = torch.topk(costs, 10, largest=False)
mappings[indices], log_probs[indices], costs[indices]

(tensor([[ 0,  1,  3,  2,  4,  6,  5, 15,  8, 11,  7, 10, 13, 12,  9, 14],
         [ 0,  1,  3,  2,  4,  6,  5, 14,  8, 11,  7, 10, 13, 12,  9, 15],
         [ 0,  2,  3,  1,  4,  6,  5,  7, 11,  8, 10,  9, 13, 12, 14, 15],
         [ 0,  1,  3,  8,  2,  4,  5,  7, 11,  6, 10,  9, 13, 12, 14, 15],
         [ 0,  1,  3,  2,  4,  6,  5,  7, 12, 11,  8,  9, 13, 10, 14, 15],
         [ 0,  2,  3,  1,  4,  6,  5,  7, 11,  8,  9, 10, 13, 12, 14, 15],
         [ 0,  1,  3,  2,  4,  6,  5,  7, 11, 15,  8,  9, 13, 12, 10, 14],
         [ 0,  1,  3,  2,  4,  6,  5,  7, 11, 15, 10,  8, 13, 12, 14,  9],
         [ 0,  1,  3,  2,  4,  6,  5, 14,  8, 11,  7, 10,  9, 12, 13, 15],
         [ 0,  1,  3,  2,  4,  6,  5, 15,  8, 11,  7, 10,  9, 12, 13, 14]],
        device='cuda:0'),
 tensor([-27.1125, -26.5833, -19.3564, -27.8346, -23.9614, -26.5911, -18.8813,
         -22.7898, -25.4051, -26.0153], device='cuda:0',
        grad_fn=<IndexBackward0>),
 tensor([6627., 6627., 6910., 6976., 6994., 7001., 7

In [22]:
mappings[log_probs.argmax()], log_probs[log_probs.argmax()], costs[log_probs.argmax()]

(tensor([ 0,  1,  3,  2,  4,  6,  5,  7, 11,  8, 10,  9, 13, 12, 14, 15],
        device='cuda:0'),
 tensor(-0.0034, device='cuda:0', grad_fn=<SelectBackward0>),
 tensor(7160., device='cuda:0', dtype=torch.float64))

In [43]:
mpnn_ptr.decoding_type = 'sampling'
mappings, log_probs = mpnn_ptr(data, num_samples=num_samples)
costs = communication_cost_multiple_samples(data.edge_index, data.edge_attr, data.batch, distance_matrix, mappings, num_samples=num_samples)
mappings[costs.argmin()], log_probs[costs.argmin()], costs[costs.argmin()]

(tensor([ 0,  1,  3,  2,  4,  6,  5,  7, 11,  8, 10,  9, 13, 12, 14, 15],
        device='cuda:0'),
 tensor(-0.0034, device='cuda:0', grad_fn=<SelectBackward0>),
 tensor(7160., device='cuda:0', dtype=torch.float64))

In [44]:
mappings[log_probs.argmax()], log_probs[log_probs.argmax()], costs[log_probs.argmax()]

(tensor([ 0,  1,  3,  2,  4,  6,  5,  7, 11,  8, 10,  9, 13, 12, 14, 15],
        device='cuda:0'),
 tensor(-0.0034, device='cuda:0', grad_fn=<SelectBackward0>),
 tensor(7160., device='cuda:0', dtype=torch.float64))